In [1]:
import sys
import os

# Assumes the current working directory is the 'usage' folder in this repo. 
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

# Internal imports. 
from blue_rov import Params as blue # blue_rov.py in main folder. 
from diffUV import dyn_body, kin, dyned_eul, dyned_quat # names defined in __init__.py. 
from diffUV.utils.operators import cross_pO, coriolis_lag_param 
from diffUV.utils.symbols import *
from diffUV.utils.quaternion_ops import euler2q, q2euler
from diffUV import simulator # names defined in __init__.py. 
from diffUV import control # names defined in __init__.py. 
import pandas as pd
# External imports. 
import casadi as ca
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

import matplotlib.pyplot as plt

In [2]:
# body representaion
uv_body = dyn_body()

# ned representaion
uv_ned = dyned_eul()

# quaternion representation
uv_quat = dyned_quat()

# kinematics representation
Kinematics = kin()

In [ ]:
def lumped_params_from_raw():
    I_x, I_y, I_z, I_xz = ca.vertsplit(I_o)
    X_du, Y_dv, Z_dw, K_dp, M_dq, N_dr = ca.vertsplit(decoupled_added_m)
    X_dq, Y_dp, M_du, K_dv = ca.vertsplit(coupled_added_m)
    return ca.vertcat(m-X_du, m-Y_dv, m-Z_dw, m*z_g-X_dq, -(m*z_g+Y_dp), -(m*z_g+K_dv), m*z_g-M_du, I_x-K_dp, I_y-M_dq, I_z-N_dr , 
                      W, B, x_g*W - x_b*B, y_g*W - y_b*B, z_g*W - z_b*B,
                      X_u,Y_v,Z_w,K_p,M_q,N_r, X_uu,Y_vv,Z_ww,K_pp,M_qq,N_rr
                      )

beta_sym = lumped_params_from_raw()